# DatasetList

In [ ]:
from caits.dataset._dataset3 import CaitsArray, DatasetList
from caits.filtering import filter_butterworth

## Dataset

In [ ]:
from caits.loading import csv_loader

data = csv_loader("data/GestureSet_small")

In [ ]:
X, y, id = data["X"], data["y"], data["id"]
caitsX = [CaitsArray(values=x.values, axis_names={
    "axis_1": {
        col: i for i, col in enumerate(x.columns)
    }
}) for x in X]
type(caitsX[0]), type(y[0]), type(id[0])

In [ ]:
datasetListObj = DatasetList(caitsX, y, id)
datasetListObj

In [ ]:
len(datasetListObj)

## Indexing

In [ ]:
datasetListObj[3]

In [ ]:
datasetListObj[3:15]

In [ ]:
datasetListObj[[3,8,16,107]]

In [ ]:
datasetListObj[1, 4]

In [ ]:
tmp = datasetListObj[1, 2:5]
tmp, tmp.X[0].shape

In [ ]:
tmp = datasetListObj[1, [3,4]]
tmp, tmp.X[0].shape

In [ ]:
datasetListObj.X[0].axis_names["axis_1"]

In [ ]:
tmp = datasetListObj[1, "acc_x_axis_g"]
tmp, tmp.X[0].shape, tmp.X[0], tmp.y, tmp._id

In [ ]:
tmp = datasetListObj[1, ["acc_x_axis_g", "acc_z_axis_g"]]
tmp, tmp.X[0].shape

In [ ]:
tmp = datasetListObj[1, "acc_x_axis_g":"gyr_x_axis_deg/s"]
tmp, tmp.X[0].shape, tmp.X[0]

In [ ]:
datasetListObj[1:4, 1]

In [ ]:
datasetListObj[1:4, 3:5]

In [ ]:
datasetListObj[1:4, [1,5]]

In [ ]:
datasetListObj[1:4, "acc_x_axis_g"]

In [ ]:
datasetListObj[1:4, ["acc_z_axis_g", "gyr_z_axis_deg/s"]]

In [ ]:
tmp = datasetListObj[1:4, "acc_x_axis_g":"gyr_x_axis_deg/s"]
tmp, tmp.X[0].shape, tmp.X[0]

In [ ]:
tmp1 = datasetListObj[:100, "acc_x_axis_g":"acc_z_axis_g"]
tmp2 = datasetListObj[:100, "gyr_x_axis_deg/s":"gyr_y_axis_deg/s"]
len(tmp1), len(tmp2), tmp1.X[0].shape, tmp2.X[0].shape, {axis: len(names) for axis, names in tmp1.X[0].axis_names.items()}, {axis: len(names) for axis, names in tmp2.X[0].axis_names.items()}

In [ ]:
axis_names = {**tmp1.X[0].axis_names["axis_1"], **tmp2.X[0].axis_names["axis_1"]}
axis_names

In [ ]:
tmp = tmp1.unify([tmp2], axis=1)
tmp, tmp.X[0].shape, tmp.X[0]

In [ ]:
tmp1 = datasetListObj[:100, ["acc_x_axis_g"]]
tmp2 = datasetListObj[:100, ["acc_y_axis_g"]]
tmp3 = datasetListObj[:100, ["acc_z_axis_g", "gyr_z_axis_deg/s"]]
tmp1.X[0], tmp2.X[0], tmp3.X[0]

In [ ]:
tmp = tmp1.unify([tmp2, tmp3], axis_names={"axis_1": {"col1": 0, "col2": 1, "col3": 2, "col4": 3}}, axis=1)
tmp, tmp.X[0].shape, tmp.X[0].axis_names

In [ ]:
tmp[:, ["col1", "col3"]].X

In [ ]:
for i, row in enumerate(datasetListObj):
    print(i)

In [ ]:
for i, batch in enumerate(datasetListObj.batch(10)):
    print(batch)

In [ ]:
train_obj, test_obj = datasetListObj.train_test_split()

In [ ]:
len(train_obj), len(test_obj)

In [ ]:
train_obj.X

In [ ]:
newDatasetListObj = train_obj + test_obj
len(newDatasetListObj)

In [ ]:
train_obj, test_obj = datasetListObj.train_test_split(random_state=42)
len(train_obj), len(test_obj)

In [ ]:
newDatasetListObj = train_obj + test_obj
len(newDatasetListObj)

In [ ]:
datasetListObj.apply(filter_butterworth, fs=200, filter_type='lowpass', cutoff_freq=50)